# The data

   ##  About the data
The analysis seeks to establish transformation of word into vectors on any text. We are not concerned about whether the text data has label or not. The data set supplied consists of  **50000 IMDB reviews**  with review ID on a certain movie  with no labels.We'll use this unlabelled data to train a model. which can be applied on test data.

Please visit the site to download the data
https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [1]:
import numpy as np
import pandas as pd

## Import the data

The data was imported from local repository using the command below.

In [2]:
from google.colab import files
files.upload()

Saving unlabeledTrainData_lyst8653.tsv to unlabeledTrainData_lyst8653.tsv


In [6]:
df=pd.read_csv("unlabeledTrainData_lyst8653.tsv",delimiter="\t",quoting=3,header=0)

In [7]:
df.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [8]:
import re,string

##  Data Cleaning
We've gone through the reviews & detected punctuations in many reviews.The punctuations don't contribute anything to our analysis & moreover they are considered as unique word & distort the meaning of other words.This is why the data needs to be cleaned before we jump into core analysis.

In [9]:
def clean_string(string):                                                         # The entire document is cleaned defining clean_string
  try:
    string=re.sub(r'^https?:\/\/<>.*[\r\n]*','',string,flags=re.MULTILINE)
    string=re.sub(r"[^A-Za-z]"," ",string)
    words=string.strip().lower().split()
    return " ".join(words)
  except:
    return " "

Above we defined a function called **clean_string** & this function we have applied on the raw review column and created a new column(**clean_review**) to save the cleaned reviews.

In [10]:
df['clean_review']=df.review.apply(clean_string)                                  # Finally cleaned format is applied on the reviews


In [11]:
print ("No.of samples \n:",(len(df)))
df.head()

No.of samples 
: 50000


,id,review,clean_review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was...",watching time chasers it obvious that it was m...
1,"""45057_0""","""I saw this film about 20 years ago and rememb...",i saw this film about years ago and remember i...
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B...",minor spoilers br br in new york joan barnard ...
3,"""7161_0""","""I went to see this film with a great deal of ...",i went to see this film with a great deal of e...
4,"""43971_0""","""Yes, I agree with everyone on this site this ...",yes i agree with everyone on this site this mo...


If we look at the data now, we'll not notice any punctuations in the **clean_review** column.


#  Word2Vec with Gensim(The Word2Vec toolkit)

Gensim is an open source Python library for natural language processing, with a focus on topic modeling.Gensim was developed and is maintained by the Czech natural language processing researcher **Radim Řehůřek** and his company RaRe Technologies.

It is not an everything-including-the-kitchen-sink NLP research library (like NLTK); instead, Gensim is a mature, focused, and efficient suite of NLP tools for topic modeling. Most notably for this tutorial, it supports an implementation of the** Word2Vec word embedding** for learning new word vectors from text.

It also provides tools for loading pre-trained word embeddings in a few formats and for making use and querying a loaded embedding.


### Objective

In this tutorial, we dig a little "deeper" into sentiment analysis. Google's Word2Vec is a deep-learning inspired method that focuses on the meaning of words. Word2Vec attempts to understand meaning and **semantic relationships** among words. It works in a way that is similar to deep approaches, such as recurrent neural nets or deep neural nets, but is computationally more efficient. This tutorial focuses on Word2Vec for sentiment analysis.

**Please install & import the gensim everytime you work on Google colab**

In [12]:
!pip install gensim --quiet     

In [13]:
import gensim

**Since we are going to work with words, so we are required to split the each review so that we can have word tokens.**

In [14]:
Document=[]
for doc in df['clean_review']:
  Document.append(doc.split(' '))  

In [15]:
len(Document)

50000

**Let us explore split reviews**

In [16]:
Document[10][6:13]     

['movie', 'i', 'am', 'not', 'sure', 'whether', 'i']

In [17]:
print(len(Document[10]))                                                          # Lenth of the 10th document ,  It has 524 words in it
print(Document[10])

524
['after', 'reading', 'the', 'comments', 'for', 'this', 'movie', 'i', 'am', 'not', 'sure', 'whether', 'i', 'should', 'be', 'angry', 'sad', 'or', 'sickened', 'seeing', 'comments', 'typical', 'of', 'people', 'who', 'a', 'know', 'absolutely', 'nothing', 'about', 'the', 'military', 'or', 'b', 'who', 'base', 'everything', 'they', 'think', 'they', 'know', 'on', 'movies', 'like', 'this', 'or', 'on', 'cnn', 'reports', 'about', 'abu', 'gharib', 'makes', 'me', 'wonder', 'about', 'the', 'state', 'of', 'intellectual', 'stimulation', 'in', 'the', 'world', 'br', 'br', 'at', 'the', 'time', 'i', 'type', 'this', 'the', 'number', 'of', 'people', 'in', 'the', 'us', 'military', 'million', 'on', 'active', 'duty', 'with', 'another', 'almost', 'in', 'the', 'guard', 'and', 'reserves', 'for', 'a', 'total', 'of', 'roughly', 'million', 'br', 'br', 'the', 'number', 'of', 'people', 'indicted', 'for', 'abuses', 'at', 'at', 'abu', 'gharib', 'currently', 'less', 'than', 'br', 'br', 'that', 'makes', 'the', 'total',

In [18]:
import logging  

In [19]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model=gensim.models.Word2Vec(Document,                                           # List of reviews
                          min_count=10,                                          # we want words appearing atleast 10 times in the vocab otherwise ignore 
                          workers=4,                                             # Use these many worker threads to train the model (=faster training with multicore machines
                           size=50,                                              # it means aword is represented by 50 numbers,in other words the number of neorons in hidden layer is 50 
                          window=5)                                              # 5 neighbors on the either side of a word

2020-09-23 09:46:57,375 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-09-23 09:46:57,380 : INFO : collecting all words and their counts
2020-09-23 09:46:57,384 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-09-23 09:46:57,867 : INFO : PROGRESS: at sentence #10000, processed 2399440 words, keeping 51654 word types
2020-09-23 09:46:58,410 : INFO : PROGRESS: at sentence #20000, processed 4835846 words, keeping 69077 word types
2020-09-23 09:46:58,893 : INFO : PROGRESS: at sentence #30000, processed 7267977 words, keeping 81515 word types
2020-09-23 09:46:59,379 : INFO : PROGRESS: at sentence #40000, processed 9669772 words, keeping 91685 word types
2020-09-23 09:46:59,850 : INFO : collected 100479 word types from a corpus of 12084660 raw words and 50000 sentences
2020-09-23 09:46:59,852 : INFO : Loading a fresh vocabulary
2020-09-23 09:47:00,236 : INFO : effective_min_count=10 retains 28322 unique words (28% of ori

**Please note that after applying Word2Vec function on the clean_review giving all the arguments corretly we have got 28322 words**

In [20]:
print(len(model.wv.vocab)) 

28322


**Let's check the dimension of a vector i.e. the number of words that represent a word**

In [21]:
print(model.wv.vector_size)  

50


In [22]:
model.wv.vectors.shape 

(28322, 50)

### Let's explore some interesting results of word2vec experiment

In [23]:
model.wv.most_similar("beautiful")   # 10 similar words beautiful,the maximum similarity is 1,minimum is 0.When they are completely similar the 
                                                                                  # Value will be 1 , when completely dissimilar,the value will be 0.

2020-09-23 09:50:22,141 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('gorgeous', 0.8498616218566895),
 ('lovely', 0.813451886177063),
 ('stunning', 0.8121933937072754),
 ('wonderful', 0.7573546171188354),
 ('haunting', 0.7496060132980347),
 ('breathtaking', 0.7036622762680054),
 ('touching', 0.6950976848602295),
 ('fabulous', 0.6866289973258972),
 ('delightful', 0.6850804090499878),
 ('magnificent', 0.6806464791297913)]

In [24]:
model.wv.most_similar("princess")                                                  # 10 similar words returned with numbers

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('prince', 0.848441481590271),
 ('widow', 0.8242537379264832),
 ('maid', 0.8132736086845398),
 ('nurse', 0.8021011352539062),
 ('servant', 0.7945051789283752),
 ('belle', 0.7807425260543823),
 ('mistress', 0.7736099362373352),
 ('katie', 0.7735347747802734),
 ('maria', 0.7725345492362976),
 ('connie', 0.7707166075706482)]

In [25]:
model.wv.doesnt_match("she talked to me in the evening publicly".split())         # publicly does not match in the sentence given

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'publicly'

Below the word **right** is represented by a dense 50 dimensional vector

In [26]:
model.wv["right"]                                                                  # right word is represented by 50 numbers in other words the word "right" is vector of 50 numbers
                                                                                   # 50 numbers are summarized weights because these numbers are obtained in the hidden layer of predefined 50 neurons

array([ 0.58482987,  0.37276646,  4.3139997 ,  1.1162901 , -3.2966876 ,
       -1.0363513 ,  1.6049155 , -1.1897538 , -0.51993334,  0.6361889 ,
       -0.7960794 ,  1.365439  ,  2.5716636 , -1.101285  ,  0.96635634,
       -0.06711897,  1.3096476 ,  0.9245065 , -1.2487943 , -1.5867445 ,
        0.96848917, -0.17020777,  1.1714509 ,  0.97659427, -0.34997207,
        0.5747086 ,  0.16904445,  0.06621201, -0.78070676, -0.4459674 ,
       -1.8318795 , -0.63698816, -0.52165806, -1.490284  ,  1.4324033 ,
        1.0249132 ,  0.5278931 , -0.56304586, -2.454342  ,  2.0688403 ,
       -1.7150204 ,  0.703759  ,  1.4021444 ,  0.8832156 ,  1.7617977 ,
       -1.468824  ,  1.0558372 , -0.87220806, -0.73403037, -0.68236196],
      dtype=float32)

In [27]:
model.wv['great']

array([-0.9054477 , -0.7499229 ,  4.01928   ,  0.7254148 , -0.98956865,
        1.188865  ,  0.27326736, -2.2435625 , -1.5555571 ,  1.442838  ,
        0.6180451 ,  1.4773593 ,  2.3255615 , -0.3421548 ,  2.3044271 ,
        0.8625231 ,  2.224525  ,  1.2045219 ,  0.80200946, -1.3115244 ,
        0.98753154, -0.0264725 , -0.586872  ,  0.17582227, -0.01529712,
       -2.0365381 ,  0.0304117 , -2.8428738 , -3.2359104 , -1.7251419 ,
        0.71908975, -0.00965737,  1.4087237 , -0.56708026, -0.78146267,
        0.78369546, -3.476549  ,  2.323809  , -0.38289136, -0.39009288,
        0.66047275,  0.12427082,  4.268108  , -1.9506902 ,  4.217624  ,
       -2.2324438 , -2.4205093 , -2.106239  ,  2.9360704 , -1.6193236 ],
      dtype=float32)

## Saving the model

In [29]:
model.save("word2vec movie-50")                                                    # We save this model for further use.
                                                                                   # Google has such many pre-trained models

2020-09-23 09:53:25,669 : INFO : saving Word2Vec object under word2vec movie-50, separately None
2020-09-23 09:53:25,671 : INFO : not storing attribute vectors_norm
2020-09-23 09:53:25,673 : INFO : not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-09-23 09:53:25,855 : INFO : saved word2vec movie-50


# Sentiment Analysis with pre-trained Word2Vec model 


## Overview
In this tutorial we'll do Sentiment analysis based on the concept of Word2Vec using our **pre-trained model** with unlabelled data where we've applied **Word2Vec** technique i.e representing a word with a dense vector of **50 numbers**. The unlabelled data has **50000 IMDB movie reviews** & we extracted  some **28000+** unique words after doing some data preprocessing & applying Word2Vec technique with length of 50 numbers.

Set the seed

In [30]:
np.random.seed(42)

###Load data
Data can be downloaded from Kaggle -> https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [31]:
files.upload()

Saving labeledTrainData_lyst3830.tsv to labeledTrainData_lyst3830.tsv


In [32]:
df1 = pd.read_csv('labeledTrainData_lyst3830.tsv',  #filepath
                 header=0, delimiter="\t", quoting=3)

print(df1.shape)  

(25000, 3)


## About the data

The labelled data set contains 25000 reviews with label(**Sentiment**). The output column  Sentiment consists of 2 categories[0 & 1]. 

**0 -- Indicates negative sentiment **               ,  if the rating < 5

**1-- Indicates positive sentiment **                  , if the rating >= 7

In [33]:
df1.iloc[10:15,:]                                                                  # Have 10th & 11th review of the dataset alongwith review id, sentiment.

,id,sentiment,review
10,"""2486_3""",0,"""What happens when an army of wetbacks, towelh..."
11,"""6811_10""",1,"""Although I generally do not like remakes beli..."
12,"""11744_9""",1,"""\""Mr. Harvey Lights a Candle\"" is anchored by..."
13,"""7369_1""",0,"""I had a feeling that after \""Submerged\"", thi..."
14,"""12081_1""",0,"""note to George Litman, and others: the Myster..."


# Data Preprocessing

**1.Split Data into Training and Test Data**

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df1['review'],
    df1['sentiment'],
    test_size=0.2, 
    random_state=42
)

**2.Build Tokenizer to get Number sequences for Each review**

In [35]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

#Vocab size
top_words = 10000

t = Tokenizer(num_words=top_words)
t.fit_on_texts(X_train.tolist())

#Get the word index for each of the word in the review
X_train = t.texts_to_sequences(X_train.tolist())
X_test = t.texts_to_sequences(X_test.tolist())

**3.Pad sequences to make each review size equal Get the word index for each of the word in the review**

We  want to bring all the reviewa into same length because we want to build matrix with this dimension

In [36]:
from tensorflow.python.keras.preprocessing import sequence

#Each review size
max_review_length = 300

X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post') 

## Build Embedding Matrix from Pre-Trained Word2Vec model

In [37]:
#Install gensim
!pip install gensim --quiet

#Load pre-trained model
import gensim
word2vec = gensim.models.Word2Vec.load('word2vec movie-50')

#Embedding Length
embedding_vector_length = word2vec.wv.vectors.shape[1]

print('Loaded word2vec model..')
print('Model shape: ', word2vec.wv.vectors.shape)

2020-09-23 10:14:16,372 : INFO : loading Word2Vec object from word2vec movie-50
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-09-23 10:14:16,519 : INFO : loading wv recursively from word2vec movie-50.wv.* with mmap=None
2020-09-23 10:14:16,520 : INFO : setting ignored attribute vectors_norm to None
2020-09-23 10:14:16,521 : INFO : loading vocabulary recursively from word2vec movie-50.vocabulary.* with mmap=None
2020-09-23 10:14:16,522 : INFO : loading trainables recursively from word2vec movie-50.trainables.* with mmap=None
2020-09-23 10:14:16,524 : INFO : setting ignored attribute cum_table to None
2020-09-23 10:14:16,526 : INFO : loaded word2vec movie-50


Loaded word2vec model..
Model shape:  (28322, 50)


In [38]:
word2vec.wv.vector_size

50

**Build matrix for current data**

In [39]:
#Initialize embedding matrix to all zeros
embedding_matrix = np.zeros((top_words + 1, # Vocablury size + 1,, we add 1 to vocab size for padding
                             embedding_vector_length))

#Steps for populating embedding matrix

#1. Check each word in tokenizer vocablury to see if it exist in pre-trained
# word2vec model.
#2. If found, update embedding matrix with embeddings for the word 
# from word2vec model

for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > top_words:
        break
    if word in word2vec.wv.vocab:
        embedding_vector = word2vec.wv[word]
        embedding_matrix[i] = embedding_vector

In [40]:
#Check embeddings for word 'great'
embedding_matrix[t.word_index['great']]

array([-0.90544772, -0.74992287,  4.01927996,  0.72541481, -0.98956865,
        1.18886495,  0.27326736, -2.24356246, -1.55555713,  1.44283795,
        0.61804509,  1.47735929,  2.32556152, -0.3421548 ,  2.30442715,
        0.86252308,  2.22452497,  1.20452189,  0.80200946, -1.31152439,
        0.98753154, -0.0264725 , -0.58687198,  0.17582227, -0.01529712,
       -2.03653812,  0.0304117 , -2.84287381, -3.23591042, -1.72514188,
        0.71908975, -0.00965737,  1.40872371, -0.56708026, -0.78146267,
        0.78369546, -3.47654891,  2.32380891, -0.38289136, -0.39009288,
        0.66047275,  0.12427082,  4.26810789, -1.95069015,  4.21762419,
       -2.23244381, -2.42050934, -2.10623908,  2.93607044, -1.61932361])

## Build the Graph

In [41]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten

#Build a sequential model
model1 = Sequential()

**Add Embedding layer**

In [42]:
model1.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length=max_review_length,
                    weights=[embedding_matrix],                                    # Pre-trained embedding
                    trainable=False)                                               # We do not want to change embedding
         )

Output from Embedding is 3 dimension 
- batch_size x max_review_length x embedding_vector_length. 

We need to flatten the output for Dense layer

In [43]:
#Flatten embedding layer output and flatten layers
model1.add(Flatten())                                                             # Flatten enables us to bring down the dimension of the prepared data
model1.add(Dense(200,activation='relu'))                                          # Dense layer is for fully connected layer
model1.add(Dense(100,activation='relu'))
model1.add(Dropout(0.5))                                                          # Dropout is required to avoid overfiting & make the model generalize
model1.add(Dense(60,activation='relu'))
model1.add(Dropout(0.4))
model1.add(Dense(30,activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))                                         # We've used sigmoid because output variable is binary

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [44]:
#from keras.utils import to_categorical
#Y_train=to_categorical(y_train,2)
#Y_test=to_categorical(y_test,2)

## Execute the graph

Here we'll  use split data to find train & validation accuracy with 10 iterations on 20000 train data & 5000 validation data with batch size of 200.

In [45]:
model1.fit(X_train,y_train,
          epochs=5,
          batch_size=200,          
          validation_data=(X_test, y_test))

Epoch 1/5
100/100 [==============================] - 2s 15ms/step - loss: 0.7320 - accuracy: 0.5313 - val_loss: 0.6431 - val_accuracy: 0.6454
Epoch 2/5
100/100 [==============================] - 1s 14ms/step - loss: 0.5938 - accuracy: 0.6896 - val_loss: 0.5322 - val_accuracy: 0.7400
Epoch 3/5
100/100 [==============================] - 1s 14ms/step - loss: 0.4553 - accuracy: 0.7998 - val_loss: 0.5038 - val_accuracy: 0.7602
Epoch 4/5
100/100 [==============================] - 1s 14ms/step - loss: 0.3524 - accuracy: 0.8548 - val_loss: 0.5250 - val_accuracy: 0.7620
Epoch 5/5
100/100 [==============================] - 1s 14ms/step - loss: 0.2466 - accuracy: 0.9024 - val_loss: 0.6024 - val_accuracy: 0.7542


In [46]:
model1.predict(X_test[10:12])

array([[0.40556923],
       [0.02359332]], dtype=float32)

In [47]:
df1.iloc[10:12,:]

,id,sentiment,review
10,"""2486_3""",0,"""What happens when an army of wetbacks, towelh..."
11,"""6811_10""",1,"""Although I generally do not like remakes beli..."
